# Método de Euler

**Objetivo general**
- Realizar el seguimiento de un conjunto de partículas dentro de un flujo usando el Método de Euler hacia adelante.

**Objetivos particulares**
- Entender en qué consiste el flujo de Poiseuille.
- Calcular la trayectoria de una partícula.
- Calcular la trayectoria de varias partículas.
- Expresar el problema en términos de un Problema de Valor Inicial.

## Contenido
- [1. Flujo en una tubería.](#1)
- [2. Cálculo de una sola trayectoria.](#2)
    - [2.1 Definición y graficación del flujo. ](#2-1)
    - [2.2 Definición de la posición inicial de la partícula y graficación. ](#2-2)
    - [2.3 Seguimiento de la partícula y graficación de la trayectoria. ](#2-3)
        - [Ejercicio 1. Implementación del Método de Euler en 2D.](#ej-1)
        - [Ejercicio 2. Implementación del Método de Euler para varios pasos de tiempo.](#ej-2)
- [3. Cálculo de varias trayectorias.](#3)
    - [3.1 Definición las posiciones iniciales. ](#3-1)
    - [3.2 Seguimiento de todas las partículas. ](#3-2)
        - [Ejercicio 3. Implementación del Método de Euler para varias partículas.](#ej-3)
        - [Ejercicio 4. Interactivo.](#ej-4)
- [4. Problemas de valor inicial.](#4)
- [5. Deducción de Método de Euler (OPCIONAL).](#5)

<a name='1'></a>
## Flujo en una tubería.
De acuerdo con la <a href="https://es.wikipedia.org/wiki/Ley_de_Poiseuille#C%C3%A1lculo_de_la_f%C3%B3rmula">Ley de Poiseuille</a> la velocidad de un líquido incompresible y uniformemente viscoso (<a href="https://es.wikipedia.org/wiki/Fluido_newtoniano">fluido newtoniano</a>) a través de un tubo cilíndrico de sección circular constante se puede calcular mediante la siguiente fórmula

$$
u(r) =\left( \dfrac{\Delta p}{4 L \mu} \right)(R^2 - r^2) \tag{1}
$$

donde $\Delta p$ representa la diferencia de presión entre dos puntos a lo largo del eje del cilíndro, $L$ es la longitud, $\mu$ es la viscosidad dinámica, $R$ es el radio del cilíndro y $r$ es el radio medido desde el eje del cilíndro, véase la siguiente figura:

<a title="Af3 at the Spanish language Wikipedia, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Poiseuille.png"><img width="300" alt="Poiseuille" src="https://upload.wikimedia.org/wikipedia/commons/4/46/Poiseuille.png"></a>     

Observamos que la fórmula $(1)$ representa una parábola. Veamos como se ve esta velocidad en una gráfica:

In [ ]:
# Importamos las bibliotecas necesarias
import matplotlib.pyplot as plt
import numpy as np
from macti.visual import plotFlujo
from macti.evaluacion import Evalua
ȩ = Evalua('MetodoEuler')

Definimos algunos datos para la fórmula

In [ ]:
ΔP = 0.298  # Diferencia de presiones
μ = 1.05e-3 # Viscosidad del flujo
L = 3.0     # Longitud del tubo
R = 0.5     # Radio del tubo
print('Diferencia de presiones: {} \nViscosidad: {} \nLongitud: {} \nRadio: {}'.format(ΔP,μ,L,R))

En la fórmula $u(r) =\left( \frac{\Delta p}{4 L \mu} \right)(R^2 - r^2)$ observamos que la $r$ varía en la dirección $y$ de los ejes Cartesianos, véase la figura de arriba. Entonces, para dibujar la velocidad en todo el tubo definimos el siguiente conjunto de coordenadas:

In [ ]:
x = np.linspace(0,L,20)
r = np.linspace(-R,R,20)
xg, yg = np.meshgrid(x, r, indexing='ij', sparse=False)

La implementación de la fórmula $(1)$ es como sigue:

In [ ]:
u = lambda x,r: ΔP*(R**2 - r**2)/(4*L*μ)  # Flujo de Poiseuille
v = lambda x,r: r*0                       # La velocidad en dirección vertical es cero.

En el código anterior hemos usado funciones `lambda` de Python, una introducción se puede ver aquí.

In [ ]:
# Graficación del campo de velocidades (u,v)
plotFlujo(xg, yg, u, v, 'quiver', 'Flujo de Poiseuille')

<a name='2'></a>
## Cálculo de una sola trayectoria.

Supongamos que soltamos una partícula en el punto $(x_i, y_i)$ dentro del cilíndro del flujo de Poiseuille descrito arriba. **¿Hacía donde se moverá esa partícula?**

Hagamos el siguiente análisis:

1. Sabemos que la velocidad se puede calcular con la fórmula simple $v = \dfrac{d}{t}$, donde $d$ es la distancia y $t$ es el tiempo.

2. De aquí sabemos que la distancia es $d = t * v $.

3. Si mi punto inicial es $x_i$ y el final es $x_f$ entonces la distancia recorrida será $d = x_f - x_i$. 

4. Por lo tanto, combinando las fórmulas de 2. y 3., es fácil ver que el punto $x_f$ se obtiene con la siguiente fórmula:

$$
x_f = x_i +  t * v \tag{2}
$$

La ecuación $(2)$ representa el **Método de Euler** (hacia adelante) para aproximar la solución de una Ecuación Diferencial Ordinaria de primer orden.

Antes de revisar los detalles de este método veamos como podemos calcular las trayectorias de algunas partículas y dibujarlas dentro del flujo.

Resolveremos el problema paso a paso. Primero para una partícula haremos lo siguiente:

1. Definición del flujo y graficación del flujo.
2. Definición de la posición inicial de la partícula y graficación.
3. Seguimiento de la partícula y graficación de la trayectoria.

<a name='2-1'></a>
### Definición y graficación del flujo.

Este punto lo hicimos antes, aquí solo repetimos los pasos realizados

In [ ]:
# Coordenadas Cartesianas
x = np.linspace(0,L,20)
r = np.linspace(-R,R,20)
xg, yg = np.meshgrid(x, r, indexing='ij', sparse=False)

# Campo de velocidad
u = lambda x,r: ΔP*(R**2 - r**2)/(4*L*μ)  # Flujo de Poiseuille
v = lambda x,r: r*0                       # La velocidad en dirección y es cero.

# Graficación del campo de velocidades (u,v)
plotFlujo(xg, yg, u, v, 'quiver', 'Flujo de Poiseuille')

<a name='2-2'></a>
### Definición de la posición inicial de la partícula y graficación.

La posición inicial de la partícula debe estar dentro del dominio de estudio, es decir dentro de $[0,L] \times [-R, R]$.

In [ ]:
# Posición inicial de la partícula
xi = 0.1 
yi = 0.0

In [ ]:
# Graficación de la posición inicial y el flujo
plt.scatter(xi, yi, fc='C3', ec='k', alpha=0.75, s=60, zorder=5)
plotFlujo(xg, yg, u, v, 'quiver', 'Posición inicial')

<a name='2-3'></a>
### Seguimiento de la partícula y graficación de la trayectoria.

A partir de la fórmula $(2)$, que es el método de Euler, podemos calcular la siguiente posición de la partícula aplicando dicha fórmula a cada coordenada:

$$
\begin{eqnarray}
x_f & = & x_i +  t * u  \tag{3}\\
y_f & = & y_i +  t * v \\
\end{eqnarray}
$$

<a name='ej-1'></a>
#### **<font color="DodgerBlue">Ejercicio 1. Implementación del Método de Euler en 2D.</font>**

<font color="DarkBlue">Para la posición inicial $(x_i, y_i) = (0.1,0.0)$ y $t = 0.05$ calcule la posición final $(x_f, y_f)$ mediante las fórmulas dadas en $(3)$.</font>

In [ ]:
# Paso de tiempo
t = 0.05

# Cálculo de la nueva posición usando el método de Euler
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
ȩ.verifica(np.array([xf, yf]), 1)

In [ ]:
# Graficación del punto inicial, la trayectoria y el campo de velocidades
plt.scatter([xi,xf], [yi, yf], fc='C3', ec='k', alpha=0.75, s=60, zorder=5)
plt.plot([xi,xf], [yi, yf],'-k')
plotFlujo(xg, yg, u, v, 'quiver', 'Seguimiento durante un paso de tiempo')

En el cálculo anterior, se realizó el seguimiento durante **un solo paso de tiempo** y definimos el paso de tiempo igual a $0.05$. 

Para calcular varios pasos de tiempo debemos definir lo siguiente:

1. El número de pasos que deseamos calcular $N_t$.
2. Definir un arreglo para almacenar las coordenadas de la trayectoria en cada paso.
3. Realizar un ciclo de cálculo que permita ir calculando la trayectoria con el método de Euler.

El siguiente algoritmo describe los pasos a realizar:
<center>
<img src="../utils/figs/algTrayectoria01.png"  width='400px'/>
</center>

En el algoritmo anterior estamos usando $h_t$ como paso de tiempo en vez de $t$.

<a name='ej-2'></a>
#### **<font color="DodgerBlue">Ejercicio 2. Implementación del Método de Euler para varios pasos de tiempo.</font>**

<font color="DarkBlue">Tomando en cuenta el algoritmo 1, complete el código que sigue agregando las fórmulas correspondientes al Método de Euler para cada coordenada.</font>

In [ ]:
# Número de pasos de tiempo.
Nt = 5  
ht = 0.05

# Posición inicial de la partícula
xi = 0.1  
yi = 0.0

# Definimos el arreglo para almacenar las posiciones
c = np.zeros((Nt, 2))
c[0] = (xi, yi)

for n in range(1, Nt): # Ciclo para calcular las posiciones
    # YOUR CODE HERE
    raise NotImplementedError()
    c[n] = (xf, yf)         # Agregamos (xf, yf) a la lista de posiciones
    (xi, yi) = (xf, yf)     # Actualizamos (xi, yi)
    
print('Posiciones de la trayectoria\n {}'.format(c))

In [ ]:
ȩ.verifica(c, 2)

In [ ]:
# Graficación del punto inicial, la trayectoria y el campo de velocidades
plt.scatter(c[:,0], c[:,1], fc='C3', ec='k', alpha=0.75, s=60, zorder=5)
plt.plot(c[:,0], c[:,1],'-k')
plotFlujo(xg, yg, u, v, 'quiver', 'Varios pasos de tiempo')

<a name='3'></a>
## Cálculo de varias trayectorias.

Para calcular varias trayectorias haremos lo siguiente.

<a name='3-1'></a>
### Definición de las posiciones iniciales.

Primero se deben definir las posiciones iniciales de $N_p$ partículas y graficarlas:

In [ ]:
# Número de partículas
Np = 7

# Coordenadas iniciales de las partículas
px = np.zeros(Np)
py = np.linspace(-0.4, 0.4, Np) # Equiespaciadas en dirección y

print('Posiciones iniciales \n px = {} \n py = {}'.format(px, py))
# Graficación de las posiciones iniciales
for i in range(Np):
    plt.scatter(px[i], py[i], ec='k', alpha=0.75, s=50, zorder=5)
plotFlujo(xg, yg, u, v, 'quiver', 'Varias partículas')

<a name='3-2'></a>
### Seguimiento de todas las partículas.

Ahora realizamos el seguimiento de cada partícula y dibujamos su trayectoria usando el siguiente algoritmo:
<center>
<img src="../utils/figs/algTrayectoria02.png"  width='400px'/>
</center>

<a name='ej-3'></a>
#### **<font color="DodgerBlue">Ejercicio 3. Implementación del Método de Euler para varias partículas.</font>**

<font color="DarkBlue">Tomando en cuenta el algoritmo 2, complete el código siguiente para calcular las trayectorias de $7$ partículas usando como posiciones iniciales los arreglos `px = [0. 0. 0. 0. 0. 0. 0.]` y `py = [-0.4 -0.26666667 -0.13333333 0. 0.13333333 0.26666667 0.4]` calculados anteriormente. Use también los valores `Nt = 10` y `ht = 0.05`.</font>

In [ ]:
Nt = 10
ht = 0.05
t = np.zeros((Np, Nt,2))

for j in range(0, Np):
    t[j, 0, :] = (px[j], py[j]) 

for j in range(0,Np):
    (xi, yi) = t[j, 0, :] # Posición inicial de la trayectoria j
    for n in range(1, Nt): # Ciclo para calcular las posiciones
        # YOUR CODE HERE
        raise NotImplementedError()
print(' Número de pasos: {} \n Paso de tiempo: {} \n Número de partículas: {}'.format(Nt, ht, Np))

In [ ]:
ȩ.verifica(t, 3)

In [ ]:
# Graficación de todas las trayectorias y del flujo
for j in range(Np):
    plt.scatter(t[j,:,0], t[j,:,1],  ec='k', s=50, alpha=0.75, zorder=5)
    plt.plot(t[j,:,0], t[j,:,1],'-', lw=1.0)
plt.plot(t[:,:,0], t[:,:,1], '-k', lw=0.75)
plotFlujo(xg, yg, u, v, 'quiver', 'Flujo de Poiseuille')

<a name='ej-4'></a>
#### **<font color="DodgerBlue">Ejercicio 4. Interactivo.</font>**

- <font color="DarkBlue">Ejecute la siguiente celda para obtener un simulador en donde puede variar algunos parámetros para graficar un flujo y las trayectorias de varias partículas.</font>
- <font color="DarkBlue">Revise la sección [Problemas de valor inicial.](#4) y posteriormente realice abra el cuaderno <a href="DecaimientoRadioactivo.ipynb">DecaimientoRadioactivo.ipynb</a> y realice los ejercicios que ahí se piden.</font> 
- <font color="DarkBlue">Posteriormente responda las preguntas en el *quiz* **Método de Euler para la solución de un PVI** en Moodle.</font> 


In [ ]:
%run "../utils/src/zFlujo_interactive.ipynb"

<a name='4'></a>
## Problema de Valor Inicial (PVI)
El problema de calcular la posición $x_f$ a partir de una posición inicial $x_i$, una velocidad $v$ y un paso de tiempo $t$, se puede escribir en términos de un problema de valor inicial bien planteado (PVI). 

Sabemos que la velocidad es el cambio de la posición con respecto al tiempo, es decir: 

$$v(x,t) = \dfrac{d x}{d t}$$

Entonces, en términos de un PVI, el cálculo de la posición $x$ se puede escribir como sigue:

Aproximar $x(t)$ al problema:
$$
\begin{eqnarray}
\dfrac{d x}{d t} & = & v(t, x) \qquad \text{ para }  t \in [0,T_{max}] \\
x(t=0) & = & x_i \qquad \text{(posición inicial)}
\end{eqnarray}
$$

En general, en un PVI se busca la solución de una ecuación diferencial ordinaria, en donde se debe satisfacer una condición inicial y se expresa como sigue:

---
Aproximar $y(t)$ al problema:
$$
\begin{eqnarray}
\dfrac{d y}{d t} & = & f(t, y) \qquad \text{ para } a < t < b \\
y(t=a) & = & \alpha \qquad \text{(condición inicial)}
\end{eqnarray}
$$

donde la función $f(t,y)$ debe satisfacer ciertas condiciones de continuidad para que exista una solución $y(t)$ única.

---

<a name='5'></a>
## Deducción del método de Euler (hacia adelante)

El método de Euler (hacia adelante) permite obtener soluciones aproximadas a un PVI, es decir una aproximación a la función $y(t)$. Para obtener este método definimos lo siguiente:
* $N_t$: número de pasos de tiempo igualmente espaciados en $[a,b]$.
* Tamaño de paso: $h_t = \dfrac{(b−a)}{N_t}$.
* Pasos de tiempo: $t_n = a + n ∗h_t$ para $n = 0,1,...,N_t$.

Supongamos que el PVI tiene una solución única $y(t)$ y que además tiene al menos dos derivadas continuas en $[a,b],$ de tal manera que para cada $n = 0,1,2, \dots , N_t-1$ tenemos la siguiente expansión en series de Taylor:

$$
y(t_{n+1}) = y(t_n) + (t_{n+1} - t_n)  \dfrac{d y}{d t} (t_n) + \dfrac{(t_{n+1} - t_n)^2}{2} \dfrac{d^2 y}{d t^2} (\xi_n)
$$

para algún número $\xi_n \in (t_n - t_{n+1})$.

Dado que $y(t)$ satisface la ecuación diferencial del PVI y como $h_t = (t_{n+1} - t_n) $ entonces podemos escribir:

$$
\begin{equation}
y(t_{n+1}) = y(t_n) + h_t  \, f(t_n, y(t_n)) + \dfrac{h_t^2}{2} y^{\prime \prime} (\xi_n)
\end{equation}
$$

El método de Euler hacia adelante (*forward*) consiste en eliminar el último término de la ecuación anterior de tal manera que, definiendo $y_n \equiv y(t_n)$ tenemos lo siguiente

$$
\begin{eqnarray}
y_0 & = & y(a) = \alpha \qquad \text{(condición inicial)}\nonumber \\
y_{n+1} & = & y_n + h_t \, f(t_n, y_n), \text{ para } n = 0,1,2, \dots, N_t - 1 \tag{2}
\end{eqnarray}
$$

La ecuación ($2$) proporciona una aproximación a la solución del PVI en el paso $t_{n+1}$ y se conoce como la ecuación en diferencias.

Este método:

* Calcula una aproximación a la solución de primer orden: $\mathcal{O}(h_t)$
* Es explícito.
* Es barato.
* Es fácil de implementar.
* Es condicionalmente estable.